# Prospecção de Dados (Data Mining) DI/FCUL - HA3

## Third Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP:`02`

* João Martins, 62532 - Hours worked on the project: 8
* Rúben Torres, 62531 - Hours worked on the project: 8
* Nuno Pereira, 56933 - Hours worked on the project: 8




The purpose of this Home Assignment is
* Find similar items with Local Sensitivity Hashing
* Do Dimensionality Reduction

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

**NOTE 3: Name of notebook should be: HA3_GROUP-XX.ipynb (where XX is the group number)**



## 1. Read the Dataset

The dataset correspond to about 99% of the Human Proteome (set of known Human Proteins - about 19,500), coded with specific structural elements. They are presented in a dictionary where the key is the [UniprotID](https://www.uniprot.org/) of the protein and the value is a set of indices of a specific structural characteristic

Students can use one of two datasets, that are **not** subsets of each other: 
* `data_d3.pickle` - smaller set of structural features (2048)
* `data_d4.pickle` - much larger set of structural features (20736) **Note:** This dataset has been Zipped to fit into moodle. Students should unzip it before usage 

Select **one** of the datasets and perform all analyses with it. 

It may be adviseable the usage of sparse matrices, especially for the `d4` dataset



In [ ]:
### Your code Here
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from itertools import combinations

#'A0A024R1R8', 'A0A024RBG1', 'A0A075B6H7', 'A0A075B6H8', 'A0A075B6H9', 'A0A075B6I0', 'A0A075B6I1'

# human_proteins_dataset= pd.read_pickle("data_d3.pickle")
human_proteins_dataset: dict = pickle.load(open("data_d3.pickle", "rb"))
dataframe = pd.DataFrame(
    list(human_proteins_dataset.items()), columns=["Id", "Proteome"]
)

# print(human_proteins_dataset.keys())

# for protein in dataframe["Proteome"][:20]:
#     print(protein)
#     print(len(protein))

all_protein = set(
    protein for proteins in human_proteins_dataset.values() for protein in proteins
)

print(f"all proteins: {all_protein}")

all_cods_protein = [key for key in human_proteins_dataset.keys()]

print(all_cods_protein)

group_protein_sets = [
    set(human_proteins_dataset[protein]) for protein in human_proteins_dataset
]

print(f"words {group_protein_sets[:2]}")

# pixa = dataframe["Proteome"][15]
# print(sorted(pixa))

# dataframe[15:]

## 2. Perform Local Sensitivity Hashing (LSH)

* examine the selected dataset in terms of similarities and select a set of LSH parameters able to capture the most similar proteins
* Comment your results

**BONUS POINTS:** It might be interesting to identify **some** of the candidate pairs in Uniprot, to check if they share some of the same properties (e.g. for [protein P28223](https://www.uniprot.org/uniprotkb/P28223/entry))


In [ ]:
def MakeBucketsT(TDocs, perms, N, B, R, NB):
    Buckets = {}
    all_docs = set(range(N))
    for b in range(B):
        SIGS = np.zeros((N, R), dtype="int32")  # initializes line sig
        for r in range(R):
            perm = perms[b * R + r]
            L = all_docs.copy()  # gets all docs as a set
            i = 0
            while len(L) > 0:
                elem = perm[i]  # get new element  from permutation
                docs_found = (
                    TDocs[elem] & L
                )  # get all the docs with a set bit on that elem that are still on the list
                if len(docs_found) > 0:  # if anything was found
                    SIGS[list(docs_found), r] = (
                        i  #   set the line sig to the current position from the perm
                    )
                    L = (
                        L - docs_found
                    )  #   update the current list removing the found docs
                i += 1  # update the current position
                if i == 478:  # this is the case that the document is empty
                    SIGS[list(L), r] = i  # Highly unlikely in a real data set
                    L = {}
                    # we have completed the signature for a given band,
                    # now make the hashes for each document
        for d in range(N):
            bucket = hash(tuple(SIGS[d])) % NB
            Buckets.setdefault((b, bucket), set()).add(d)
    return Buckets


def LSHTs(docs, words_len, B, R, NB=28934501, verbose=True):
    N, M = len(docs), words_len
    if verbose:
        print("transpose the data set")
    data = [[] for i in range(M)]
    for i, doc in enumerate(docs):
        for word in doc:
            data[word].append(i)
    dataT = [set(L) for L in data]
    P = B * R
    np.random.seed(3)
    if verbose:
        print(
            "Generating %d permutations for %6.3f similarity" % (P, (1 / B) ** (1 / R))
        )
    perms = [np.random.permutation(M) for i in range(P)]
    if verbose:
        print("Computing buckets...")
    buckets = MakeBucketsT(dataT, perms, N, B, R, NB)
    return buckets


def make_word_indexes(words) -> dict:
    return dict(zip(words, range(len(words))))


def make_indexed_docs(docs, word_index) -> list:
    indexed_docs = []
    for doc in docs:
        indexed_doc = set(word_index[word] for word in doc)
        indexed_docs.append(indexed_doc)
    return indexed_docs


def remove_empty_docs(words_text_sets) -> list:
    return [doc for doc in words_text_sets if len(doc) > 0]


def JaccardSimS(s1, s2):
    return len(s1 & s2) / len(s1 | s2)


def get_all_similar_docs(buckets):
    sim_docs = []
    for b, buck in buckets:
        if len(buckets[(b, buck)]) > 1:
            sim_docs += combinations(buckets[(b, buck)], 2)
    sim_docs = set(sim_docs)
    D = {}
    for i, j in sim_docs:
        D.setdefault(i, set()).add(j)
        D.setdefault(j, set()).add(i)
    return D

In [ ]:
# Process the dataset

protein_index = make_word_indexes(all_protein)
group_protein_sets = remove_empty_docs(group_protein_sets)
idx_protein = make_indexed_docs(group_protein_sets, protein_index)


# print(f"words {protein_index.keys()}")
# print(f"word_index {group_protein_sets[:10]}")
# print(f"indexed_docs {idx_protein[:10]}")

# print(sorted(idx_protein[9]))
# print(sorted(idx_protein[15]))
# print(JaccardSimS(idx_protein[9],idx_protein[15]))

# print(all_cods_protein[17636])


# common_elements = set(idx_protein[15]) == set(pixa)
# print("Common elements:", common_elements)

BANDS = 40
ROWS = 100
bucks = LSHTs(idx_protein, len(protein_index), BANDS, ROWS)

for b, buck in bucks:
    if len(bucks[(b, buck)]) > 1:
        print("Band", b, "suggests these similar docs:", bucks[(b, buck)])

In [ ]:
######## MUST BE DELETED ########
sim_docs = get_all_similar_docs(bucks)

i = 27
for j in sim_docs[i]:
    print(
        "Documents %6d, and %6d, are %7.4f similar"
        % (i, j, JaccardSimS(idx_protein[i], idx_protein[j]))
    )

### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum


## 3. Do dimensionality reduction 

Use the techniques discussued in class to make an appropriate dimensional reduction of the selected dataset. It is not necesary to be extensive, **it is better to select one approach and do it well than try a lot of techniques with poor insights and analysis**

It is important to do some sensitivity analysis, relating the dataset size reduction to the loss of information



In [ ]:
### Add supporting functions here
from sklearn.decomposition import PCA, TruncatedSVD
from scipy.sparse import dok_array, csr_array, csc_array, bsr_array, lil_array


def plot_component_representation(X, Y, xlabel, ylabel) -> None:
    plt.plot(X, Y, label="SVD")
    plt.legend()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.show()

In [ ]:
### Add processing code here
# Create a list of sets of proteins for each group
group_protein_sets = [
    set(human_proteins_dataset[protein]) for protein in human_proteins_dataset
]

print(group_protein_sets[:20])

# Step 1: Find all unique proteins across all sets
unique_proteins = sorted(
    set(protein for proteins in group_protein_sets for protein in proteins)
)

print(unique_proteins)

# Step 2: Create a mapping from proteins to column indices
protein_to_index = {protein: idx for idx, protein in enumerate(unique_proteins)}

print(protein_to_index)

# Step 3: Initialize the binary matrix
result_array = np.zeros((len(group_protein_sets), len(unique_proteins)), dtype=int)

# Step 4: Populate the binary matrix
for row_idx, proteins in enumerate(group_protein_sets):
    for protein in proteins:
        col_idx = protein_to_index[protein]
        result_array[row_idx, col_idx] = 1

# print(result_array[:20,:20])

# # Step 5: Perform SVD
u, s, v = np.linalg.svd(result_array) 

# Print the results with specified print options
np.set_printoptions(precision=4, suppress=True)

print("U shape: ", u.shape)
print(u)
print("S shape: ", s.shape)
print(s)
print("V shape: ", v.shape)
print(v)

components_number = []
consept_representation = []
for i in range(len(s)):
    # print("first %d components have a combined importance of %7.4f" %(i+1, s[:i+1].sum()/s.sum()))
    components_number.append(i)
    consept_representation.append(s[: i + 1].sum() / s.sum())

plot_component_representation(
    components_number, 
    consept_representation,
    "Number of components",
    "Combined importance of the previous components",
)

In [ ]:
print(result_array[:20,:20]) 

In [ ]:
S_csc=csc_array(result_array)

pca = TruncatedSVD(n_components=500)
pca.fit_transform(S_csc)
tve = 0
for i, ve in enumerate(pca.explained_variance_ratio_):
    tve += ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve))
print()
# print("Actual Eigenvalues:", pca.singular_values_)
for i, comp in enumerate(pca.components_):
    print("PC", i, "-->", comp)

## 4. Discuss your findings [to fill on your own]

* Comment your results above
* Discuss how could they be used for the full Uniprot that currently has about [248 Million proteins](https://www.uniprot.org/uniprotkb/statistics)


Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum
